## Application of the Bures method for chemical kinetics
The Bures method is a graphical method ([Bures, Angew. Chem. Int. Ed. 2016](https://onlinelibrary.wiley.com/doi/full/10.1002/anie.201508983), [Nielsen & Bures, Chem. Sci. 2021](https://pubs.rsc.org/en/content/articlelanding/2019/sc/c8sc04698k)) for the determination of reaction orders, employing only concentration data without the need of computing reaction rates. It relies on the overlap of different concentration curves, and thus it cannot be fully parametrized or automated. Nevertheless, it offers an easy way to determine reaction orders, and it can be easily implemented in Python.

In this project, we will consider two "variants" of the method. The first considers only the order in *catalyst*, which is easier to implement due to the catalyst concentration being constant across the kinetics. The second (also known as the VTNA - Variable Time Normalization Analysis) method is more general, and can be applied to any of the reactants, but it is a bit more complex.

### 1. Order in catalyst

There must be at least *two* data series with different initial catalyst loadings. We may then consider the concentration of any reagent or product in the dataset. For example:

<img src='Raw_profiles.png' width=400>

Here we have a case with 0.05 M and another with 0.02 M (with the reaction with more catalyst being of course faster).

The Bures method is purely *graphical*: requires to plot these concentrations at different catalyst loadings against a normalized time scale $Z = t \cdot [cat]^{r}$. The value of the exponent **r** for which all the curves fully overlap is the order in catalyst for the reaction, which is the parameter to be determined.

<img src='Scaled_profiles.png' width=400>

And this value of **r** is the order in catalyst for the reaction.

The key steps to apply this variant of the protocol are:
1. Read information from a CSV file.
2. Format and clean data: get catalyst concentration values for each column.
3. Plot the concentration vs. time profiles.
4. Try a value for **r** and plot concentration vs $t \cdot [cat]^{r}$ profiles.
5. Attempt to locate ideal **r** value.


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
#-> 1. Read information: CSV file in datasets/m1_fig2a_simple_michaelis.dat.
# Header line contains the catalyst concentrations

filename = "m1_fig2a_simple_michaelis.dat"


In [5]:
# 2. Format and clean data: get catalyst concentration values for each column.


In [6]:
# 3. Plot the concentration vs. time profiles from the DataFrame

In [7]:
# 4. Try a value for **r** (e.g. r=0.5) and plot concentration vs $t \cdot [cat]^{r}$ profiles.
# Add title, axis labels and legend

r = 0.5



In [8]:
# 5. Attempt to locate ideal **r** value, by testing several possibilities. Plot all curves in a single 
# layout, using plt.subplots. There must be as many plots as tested values.
# Hint: use a list or an array, with np.linspace




### 2. Application of the more general VTNA 
Not as straightforward as the previous one, but appliable to any reactant in the process. For each reactant whose order is to be determined, we need at least *two* data series where only the concentration of that reactant X varies, with all other variables being constant.

*Graphical* method: plot the concentrations at different initial loadings against an estimation of the time integral of the concentration of X to the power of *r*. The formula for this is:

$Z = \int_{0̣}^{n} [X]^{r} dt = \sum_{i=1}^{n} \left(\frac{[X]_{i} + [X]_{i-1}}{2}\right) (t_{i} - t_{i-1})$

When plotting any concentration in the dataset against Z, the value of **r** for which all the curves fully overlap is the order in X for the reaction.

**Important note**: we may select the concentration of any species (reactant, product...) to do the plotting against the normalized timescale **Z**. However, if the initial concentration of the selected species is not the same in the different experiments, the curves will be shifted. We may displace the curves along the y-axis, but it is simpler to just select a species to plot that is constant: in general, avoid plotting against the species *X* that has been used to compute **Z**.

Key steps:
1. Read information: multiple CSV files, with datasets and initial info.
2. Format and clean data: get the information sets for each molecules so that every parameter is constant except for the concentration of the target species.
3. Plot the concentration vs. time profiles.
4. Try a value for **r<sub>X</sub>** and plot concentration vs **Z** profiles.
5. Attempt to locate ideal **r** value.

#### Data manipulation
The **Z** term requires a few important data transformations from the raw concentration array:
1. From time, we need the *intervals* $\Delta{}t = t_{i} - t_{i-1}$.
2. For the target concentrations, we need *rolling averages* that take $c_i$ and $c_{i-1}$, to compute $0.5 (c_i + c_{i-1})$. In Pandas, there is the `.rolling()` method, which provides a *Rolling* object in which we can applicate aggregation functions. 
3. Then, we compute the product term $\Delta{}t\cdot[X]_{avg}^{r}$.
4. Finally, the *cumulative sum* of the product terms produces the final **Z**. From Pandas, there is the `.cumsum()` method.

Or, in a graphical representation:
<img src='bures_expl.png' width=800>

*Hint*: for **step 2**, you should get the corresponding *Rolling* object and then apply the function required to get *averages*.

**Step 1**: reading information. The corresponding files are:
- Array with general information (experiment index and initial concentrations of A, B  and catalyst) is at `datasets/vtna_ex1_init_vals.dat`
- Arrays with time and concentrations of A and B are at `datasets/vtna_ex1_X.dat`, with X ranging between 1 and 4 (experiment indices)

In [9]:
#-> 1. Data reading

# Read general information


# Read all individual files. For every individual array, add a column, named exp, with the corresponding experiment index 



**Step 2**: We must check which combinations have to be considered: pairs of experiments where everything is constant, except for one variable. Display the array of general information and choose.


In [ ]:
#-> 2. Show the DataFrame and write the pairs of indices for each species as variables.

**Step 3**. Plot the concentration vs. time plots for the species *P*, in a 1x3 layout, for each of the three pairs of experiments decided in the previous step. That is:

- Concentration of P against time for the pair (i,j) used to determine the order in A.
- Concentration of P against time for the pair (k,l) used to determine the order in B.
- Concentration of P against time for the pair (m,n) used to determine the order in catalyst.

In [11]:
#-> 3. Create the layout

# Build the three plots

**Step 4**. Test a single value for the exponent *r*, to determine the order in the reactant A. To do this, you will need to compute the **Z** term from the VTNA equations, considering the pair *(i,j)* selected for the species A. Follow the guidelines for data transformations explained before. Any concentration can be plotted, as it does not have to be A: here we will consider the product P.

In [12]:
#-> 4. Solve the VTNA equations.

# Get time intervals

# Get rolling averages

# Get the per-observation product: time interval x rolling average

# Compute cumulative sums of these products: normalized time scale

**Step 5**. Test several values for the exponent *r*, to determine the order in the reactant A. Repeat the previous strategy, but this time testing an array of values of r: use a 1x4 layout and test 4 values at once.

In [13]:
#-> 5. Test a set of r values for VTNA. Plot all curves in a single layout, using plt.subplots. 
# There must be as many plots as tested values.
# Hint: use a list or an array, with np.linspace 

### 3. Generalization and better practices
Once the problem has been solved (or at least we know how to do it), it is advisable to clean up and organize the corresponding code. To do this, tasks should be encapsulated in *functions*, so the code becomes easier to use.

Here, you will write a function that, taking these arguments:
- List of r values to test.
- List of DataFrames with concentration and time corresponding to a pair of experiments used to determine the order in a species X.
- Name/label of the target species X for which we are determining the order (corresponds to a column in the DataFrame).
- Name/label of the species whose concentration is to be plotted (corresponds to a column in the DataFrame but does not have to be the species for which we are computing the order).

applies the VTNA method, as done before.

In [ ]:
#-> Write the function

In [ ]:
#-> Test the function for A, B and catalyst